# Garage Data System: Dual PySpark & SQL Analysis


<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/pavanhalde21/PySpark_lecture_files/blob/main/Questions_01/garage_analysis_sql_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

In [25]:
# Cell 1: Install necessary libraries
!pip install pyspark num2words --quiet

In [26]:

# Cell 2: Create dummy CSV files for the analysis
%%writefile customer_table.csv
cid,cname,cadd,contact,creditdays,date,gender
1001,"cyona blake",NEW YORK,12345684,20,20-JAN-11,female
1002,"JOHN SMITH",NEW JERSI,1341684,20,21-FEB-11,male
1003,"JORDEN WOOD",PRAG,1805184,20,22-MAR-11,male
1004,CHRISTANA,MANHATTON,1125684,31,23-APR-13,female
1005,"TOM HILL",LONDON,1239284,10,25-JUN-15,male
1006,"KAMILA JOSEF",PRAISE,124568,9,28-JUL-11,female
1007,"ANDRU SYMON",TEXAS,125654,15,01-APR-16,male
1008,"SANJU SAMSUNG",NEW YORK,12346341,4,20-JAN-16,male


Overwriting customer_table.csv


In [27]:
%%writefile ser_det_table.csv
sid,cid,eid,spid,type_veh,veh_no,typ_ser,ser_date,qty,sp_rate,sp_amt,sp_g,ser_amt,comm,total
6001,1001,3001,4001,TWO WHEELER,MH15CA3228,TUBE DAMAGED,02-JAN-11,1.0,250,250,35,50,0,335
6002,1002,3002,4002,TWO WHEELER,MH16U5713,FULL SERVICING,04-MAR-11,1.0,400,400,52,300,50,752
6003,1003,3004,4005,TWO WHEELER,MH12PQ1313,CLUTCH WIRE,22-AUG-11,1.0,129,129,0,10,0,139
6004,1002,3002,4002,TWO WHEELER,MH16U5713,FULL SERVICING,05-MAY-11,1.0,400,400,52,300,50,752
6005,1004,3001,4009,TWO WHEELER,MH14PA335,COLOR,21-OCT-11,2.5,340,850,119,500,150,1469
6006,1006,3001,4009,TWO WHEELER,MH12WE334,COLOR,01-DEC-11,2.5,340,850,119,500,150,1469
6007,1007,3001,4001,FOUR WHEELER,MH17BB1345,TUBE DAMAGED,01-JAN-12,1.0,250,250,35,150,0,435


Overwriting ser_det_table.csv


In [28]:
%%writefile employee_table.csv
eid,ename,ejob,eadd,econtact,esal,edoj,edol
3001,"STEVEN KING",PAINTER,NEW YORK,10367454,1200,01-JAN-08,
3002,"DAVID AUSTIN",FITTER,MANAHTON,10367434,1100,19-AUG-10,
3003,"BRUCE ERENST",MECHANIC,NEW JERCY,10367264,2200,08-SEP-10,
3004,"LUIS POP",MECHANIC,NEW JERCY,10367264,1700,19-OCT-19,06-SEP-10
3005,"SHERI GOMES",FITTER,PARIS,10327264,1000,19-OCT-09,01-AUG-10
3000,"JAMES PHILIP",FITTER,PARIS,10322264,,01-JAN-08,

Overwriting employee_table.csv


In [29]:
%%writefile sparepart_table.csv
spid,spname,sprate,spunit
4001,TWO WHEELER TUBE,250,NOS
4002,TWO WHEELER ENGINE OIL,400,LTRS
4003,FOUR WHEELER ENGINE OIL,5000,LTRS
4004,TWO WHEELER CARBORATOR,680,NOS
4005,TWO WHEELER CLUTCH WIRE,129,MTRS
4006,TWO WHEELER TAIIL LIGHT,100,NOS
4007,TWO WHEELER INDICATORS,150,NOS
4008,FOUR WHEELER GASKIT,1340,NOS
4009,WHITE COLOUR,340,LTRS
4010,BLACK COLOUR,240,LTRS
4011,TWO WHEELER SIDE MIRROR,250,NOS
4012,FOUR WHEELER SIDE MIRROR,450,NOS
4013,TWO WHEELER SHOCKUP,1320,PAIR
4014,FOUR WHEELER BUMPER,6000,NOS
4015,FOUR WHEELER FRONT GLASS,7000,PCS

Overwriting sparepart_table.csv


In [30]:
%%writefile purchase_table.csv
pid,vid,spid,pqty,sprate,spgst,pdate,transcost,total,rcv_eid
5001,2001,4001,10,250,350.0,01-JAN-11,1300,4150.0,3001
5002,2002,4002,4,400,288.0,02-MAR-11,100,1988.0,3001
5003,2003,4004,8,680,761.6,12-JUN-11,50,6251.6,3003
5004,2004,4005,10,129,154.8,22-AUG-11,125,1569.8,3004
5005,2005,4006,20,100,300.0,07-SEP-11,20,2320.0,3003
5006,2006,4007,30,150,630.0,11-OCT-11,60,5190.0,3000
5007,2001,4003,20,5000,14000.0,07-SEP-11,1000,115000.0,3000
5008,2006,4005,1,129,15.48,12-OCT-11,50,194.48,3005
5009,2006,4005,1,129,15.48,15-OCT-11,50,194.48,3005
5010,2006,4009,5,340,238.0,20-OCT-11,0,1938.0,3005

Overwriting purchase_table.csv


In [31]:
%%writefile vendor_table.csv
vid,vname,vadd,vcontact,vcreditdays,vdate
2001,"KIRAN PATIL",PUNE,20535535,20,20-JAN-10
2002,"PRAKASH JAIN",MUNBAI,2063564,10,05-NOV-11
2003,"SWAPNIL THETE",NASHIK,25355352,15,18-MAR-10
2004,"AMOL SHENDE",SATARA,23674776,18,07-APR-15
2005,"KARAN SINTRE",BULDHANA,256756,30,22-APR-09
2006,"RAM KULKARNI",OSMANABAD,20367454,21,09-JUL-10

Overwriting vendor_table.csv


In [32]:
# =====================================================================
# Garage Data System: Dual PySpark & SQL Analysis
# =====================================================================

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import StringType
from num2words import num2words  # You may need to run: pip install num2words

# ---------------------------------------------------------------------
# 1. Spark Session and Data Loading
# ---------------------------------------------------------------------
spark = SparkSession.builder.appName("GarageAnalysis").getOrCreate()

# Load data into DataFrames (expects CSV files in the working directory)
customer_df = spark.read.csv("customer_table.csv", header=True, inferSchema=True)
ser_det_df = spark.read.csv("ser_det_table.csv", header=True, inferSchema=True)
employee_df = spark.read.csv("employee_table.csv", header=True, inferSchema=True)
sparepart_df = spark.read.csv("sparepart_table.csv", header=True, inferSchema=True)
purchase_df = spark.read.csv("purchase_table.csv", header=True, inferSchema=True)
vendor_df = spark.read.csv("vendor_table.csv", header=True, inferSchema=True)

# ---------------------------------------------------------------------
# 2. Create Temporary Views for Spark SQL
# ---------------------------------------------------------------------
customer_df.createOrReplaceTempView("customer_table")
ser_det_df.createOrReplaceTempView("ser_det_table")
employee_df.createOrReplaceTempView("employee_table")
sparepart_df.createOrReplaceTempView("sparepart_table")
purchase_df.createOrReplaceTempView("purchase_table")
vendor_df.createOrReplaceTempView("vendor_table")


In [111]:
print("customer_table")
customer_df.show(truncate=False)
print("ser_det_table")
ser_det_df.show(truncate=False)
print("employee_table")
employee_df.show(truncate=False)
print("sparepart_table")
sparepart_df.show(truncate=False)
print("purchase_table")
purchase_df.show(truncate=False)
print("vendor_table")
vendor_df.show(truncate=False)


customer_table
+----+-------------+---------+--------+----------+---------+------+
|cid |cname        |cadd     |contact |creditdays|date     |gender|
+----+-------------+---------+--------+----------+---------+------+
|1001|cyona blake  |NEW YORK |12345684|20        |20-JAN-11|female|
|1002|JOHN SMITH   |NEW JERSI|1341684 |20        |21-FEB-11|male  |
|1003|JORDEN WOOD  |PRAG     |1805184 |20        |22-MAR-11|male  |
|1004|CHRISTANA    |MANHATTON|1125684 |31        |23-APR-13|female|
|1005|TOM HILL     |LONDON   |1239284 |10        |25-JUN-15|male  |
|1006|KAMILA JOSEF |PRAISE   |124568  |9         |28-JUL-11|female|
|1007|ANDRU SYMON  |TEXAS    |125654  |15        |01-APR-16|male  |
|1008|SANJU SAMSUNG|NEW YORK |12346341|4         |20-JAN-16|male  |
+----+-------------+---------+--------+----------+---------+------+

ser_det_table
+----+----+----+----+------------+----------+--------------+---------+---+-------+------+----+-------+----+-----+
|sid |cid |eid |spid|type_veh    |veh_no

## <a id='q1'></a>Q.1 List all the customers serviced.

In [33]:
# Q.1 List all the customers serviced.
# -- DataFrame API Solution --
customer_df.join(ser_det_df, customer_df.cid == ser_det_df.cid, "inner").select("cname").distinct().show()
# -- Spark SQL Solution --
spark.sql("""
    SELECT DISTINCT c.cname
    FROM customer_table c
    JOIN ser_det_table s ON c.cid = s.cid
""").show()

+------------+
|       cname|
+------------+
|  JOHN SMITH|
|   CHRISTANA|
|KAMILA JOSEF|
| cyona blake|
| ANDRU SYMON|
| JORDEN WOOD|
+------------+

+------------+
|       cname|
+------------+
|  JOHN SMITH|
|   CHRISTANA|
|KAMILA JOSEF|
| cyona blake|
| ANDRU SYMON|
| JORDEN WOOD|
+------------+



## <a id='q2'></a>Q.2 Customers who are not serviced.

In [34]:
# Q.2 Customers who are not serviced.
# -- DataFrame API Solution --
customer_df.join(ser_det_df, customer_df.cid == ser_det_df.cid, "left_anti").select("cname").show()
# -- Spark SQL Solution --
spark.sql("""
    SELECT c.cname
    FROM customer_table c
    LEFT ANTI JOIN ser_det_table s ON c.cid = s.cid
""").show()

+-------------+
|        cname|
+-------------+
|     TOM HILL|
|SANJU SAMSUNG|
+-------------+

+-------------+
|        cname|
+-------------+
|     TOM HILL|
|SANJU SAMSUNG|
+-------------+



## <a id='q3'></a>Q.3 Employees who have not received the commission.

In [35]:
# Q.3 Employees who have not received the commission.
# -- DataFrame API Solution --
employee_df.join(ser_det_df, employee_df.eid == ser_det_df.eid).where(F.col("comm") == 0).select("ename").distinct().show()
# -- Spark SQL Solution --
spark.sql("""
    SELECT DISTINCT e.ename
    FROM employee_table e
    JOIN ser_det_table s ON e.eid=s.eid
    WHERE s.comm = 0
""").show()

+-----------+
|      ename|
+-----------+
|   LUIS POP|
|STEVEN KING|
+-----------+

+-----------+
|      ename|
+-----------+
|   LUIS POP|
|STEVEN KING|
+-----------+



## <a id='q4'></a>Q.4 Name the employee who have maximum Commission.

In [36]:
# Q.4 Name the employee who have maximum Commission.
# -- DataFrame API Solution --
max_comm = ser_det_df.agg(F.max("comm")).collect()[0][0]
employee_df.join(ser_det_df, employee_df.eid == ser_det_df.eid).where(F.col("comm") == max_comm).select("ename").show()
# -- Spark SQL Solution --
spark.sql("""
    SELECT e.ename
    FROM employee_table e
    JOIN ser_det_table s ON e.eid=s.eid
    WHERE s.comm = (SELECT MAX(comm) FROM ser_det_table)
""").show()

+-----------+
|      ename|
+-----------+
|STEVEN KING|
|STEVEN KING|
+-----------+

+-----------+
|      ename|
+-----------+
|STEVEN KING|
|STEVEN KING|
+-----------+



## <a id='q5'></a>Q.5 Show employee name and minimum commission amount received by an employee.

In [37]:
# Q.5 Show employee name and minimum commission amount received by an employee.
# -- DataFrame API Solution --
min_comm = ser_det_df.where(F.col("comm") > 0).agg(F.min("comm")).collect()[0][0]
employee_df.join(ser_det_df, employee_df.eid == ser_det_df.eid).where(F.col("comm") == min_comm).select("ename", "comm").show()
# -- Spark SQL Solution --
spark.sql("""
    SELECT e.ename, s.comm
    FROM employee_table e
    JOIN ser_det_table s ON e.eid = s.eid
    WHERE s.comm = (SELECT MIN(comm) FROM ser_det_table WHERE comm > 0)
""").show()

+------------+----+
|       ename|comm|
+------------+----+
|DAVID AUSTIN|  50|
|DAVID AUSTIN|  50|
+------------+----+

+------------+----+
|       ename|comm|
+------------+----+
|DAVID AUSTIN|  50|
|DAVID AUSTIN|  50|
+------------+----+



## <a id='q6'></a>Q.6 Display the Middle record from any table.

In [38]:
# Q.6 Display the Middle record from any table.
# -- DataFrame API Solution --
total_count = ser_det_df.count()
middle_row_index = (total_count // 2) + 1
window_spec = Window.orderBy("sid")
ser_det_df.withColumn("rn", F.row_number().over(window_spec)).where(F.col("rn") == middle_row_index).drop("rn").show()
# -- Spark SQL Solution --
spark.sql("""
    SELECT * FROM (
        SELECT *, ROW_NUMBER() OVER (ORDER BY sid) as rn, COUNT(*) OVER() as total_count
        FROM ser_det_table
    )
    WHERE rn = CAST(CEIL(total_count / 2.0) AS INT)
""").show(1, False)

+----+----+----+----+-----------+---------+--------------+---------+---+-------+------+----+-------+----+-----+
| sid| cid| eid|spid|   type_veh|   veh_no|       typ_ser| ser_date|qty|sp_rate|sp_amt|sp_g|ser_amt|comm|total|
+----+----+----+----+-----------+---------+--------------+---------+---+-------+------+----+-------+----+-----+
|6004|1002|3002|4002|TWO WHEELER|MH16U5713|FULL SERVICING|05-MAY-11|1.0|    400|   400|  52|    300|  50|  752|
+----+----+----+----+-----------+---------+--------------+---------+---+-------+------+----+-------+----+-----+

+----+----+----+----+-----------+---------+--------------+---------+---+-------+------+----+-------+----+-----+---+-----------+
|sid |cid |eid |spid|type_veh   |veh_no   |typ_ser       |ser_date |qty|sp_rate|sp_amt|sp_g|ser_amt|comm|total|rn |total_count|
+----+----+----+----+-----------+---------+--------------+---------+---+-------+------+----+-------+----+-----+---+-----------+
|6004|1002|3002|4002|TWO WHEELER|MH16U5713|FULL SERVICI

## <a id='q7'></a>Q.7 Display last 4 records of any table.

In [39]:
# Q.7 Display last 4 records of any table.
# -- DataFrame API Solution --
employee_df.orderBy(F.col("eid").desc()).limit(4).show()
# -- Spark SQL Solution --
spark.sql("SELECT * FROM employee_table ORDER BY eid DESC LIMIT 4").show()

+----+------------+--------+---------+--------+----+---------+---------+
| eid|       ename|    ejob|     eadd|econtact|esal|     edoj|     edol|
+----+------------+--------+---------+--------+----+---------+---------+
|3005| SHERI GOMES|  FITTER|    PARIS|10327264|1000|19-OCT-09|01-AUG-10|
|3004|    LUIS POP|MECHANIC|NEW JERCY|10367264|1700|19-OCT-19|06-SEP-10|
|3003|BRUCE ERENST|MECHANIC|NEW JERCY|10367264|2200|08-SEP-10|     NULL|
|3002|DAVID AUSTIN|  FITTER| MANAHTON|10367434|1100|19-AUG-10|     NULL|
+----+------------+--------+---------+--------+----+---------+---------+

+----+------------+--------+---------+--------+----+---------+---------+
| eid|       ename|    ejob|     eadd|econtact|esal|     edoj|     edol|
+----+------------+--------+---------+--------+----+---------+---------+
|3005| SHERI GOMES|  FITTER|    PARIS|10327264|1000|19-OCT-09|01-AUG-10|
|3004|    LUIS POP|MECHANIC|NEW JERCY|10367264|1700|19-OCT-19|06-SEP-10|
|3003|BRUCE ERENST|MECHANIC|NEW JERCY|10367264|220

## <a id='q8'></a>Q.8 Count the number of records without count function from any table.

In [40]:
# Q.8 Count the number of records without count function from any table.
# -- DataFrame API Solution --
window_spec = Window.orderBy("eid")
count_val = employee_df.withColumn("rn", F.row_number().over(window_spec)).agg(F.max("rn")).collect()[0][0]
print(f"Record count for employee_table: {count_val}")
# -- Spark SQL Solution --
spark.sql("SELECT MAX(rn) as count FROM (SELECT ROW_NUMBER() OVER (ORDER BY eid) as rn FROM employee_table)").show()

Record count for employee_table: 6
+-----+
|count|
+-----+
|    6|
+-----+



## <a id='q9'></a>Q.9 Show unique records from "Ser_det" table on cid.

In [41]:
# Q.9 Show unique records from "Ser_det" table on cid.
# -- DataFrame API Solution --
ser_det_df.dropDuplicates(["cid"]).show()
# -- Spark SQL Solution --
spark.sql("""
    SELECT * FROM (
        SELECT *, ROW_NUMBER() OVER(PARTITION BY cid ORDER BY sid) as rn
        FROM ser_det_table
    ) WHERE rn = 1
""").show()

+----+----+----+----+------------+----------+--------------+---------+---+-------+------+----+-------+----+-----+
| sid| cid| eid|spid|    type_veh|    veh_no|       typ_ser| ser_date|qty|sp_rate|sp_amt|sp_g|ser_amt|comm|total|
+----+----+----+----+------------+----------+--------------+---------+---+-------+------+----+-------+----+-----+
|6001|1001|3001|4001| TWO WHEELER|MH15CA3228|  TUBE DAMAGED|02-JAN-11|1.0|    250|   250|  35|     50|   0|  335|
|6002|1002|3002|4002| TWO WHEELER| MH16U5713|FULL SERVICING|04-MAR-11|1.0|    400|   400|  52|    300|  50|  752|
|6003|1003|3004|4005| TWO WHEELER|MH12PQ1313|   CLUTCH WIRE|22-AUG-11|1.0|    129|   129|   0|     10|   0|  139|
|6005|1004|3001|4009| TWO WHEELER| MH14PA335|         COLOR|21-OCT-11|2.5|    340|   850| 119|    500| 150| 1469|
|6006|1006|3001|4009| TWO WHEELER| MH12WE334|         COLOR|01-DEC-11|2.5|    340|   850| 119|    500| 150| 1469|
|6007|1007|3001|4001|FOUR WHEELER|MH17BB1345|  TUBE DAMAGED|01-JAN-12|1.0|    250|   250

## <a id='q10'></a>Q.10 Show the name of Customer who have paid maximum amount.

In [42]:
# Q.10 Show the name of Customer who have paid maximum amount.
# -- DataFrame API Solution --
max_total = ser_det_df.agg(F.max("total")).collect()[0][0]
customer_df.join(ser_det_df, customer_df.cid == ser_det_df.cid).where(F.col("total") == max_total).select("cname").show()
# -- Spark SQL Solution --
spark.sql("""
    SELECT c.cname
    FROM customer_table c
    JOIN ser_det_table s ON c.cid = s.cid
    WHERE s.total = (SELECT MAX(total) FROM ser_det_table)
""").show()

+------------+
|       cname|
+------------+
|   CHRISTANA|
|KAMILA JOSEF|
+------------+

+------------+
|       cname|
+------------+
|   CHRISTANA|
|KAMILA JOSEF|
+------------+



## <a id='q11'></a>Q.11 Display Employees who are not currently working.

In [43]:
# Q.11 Display Employees who are not currently working.
# -- DataFrame API Solution --
employee_df.where(F.col("edol").isNotNull()).select("ename").show()
# -- Spark SQL Solution --
spark.sql("SELECT ename FROM employee_table WHERE edol IS NOT NULL").show()

+-----------+
|      ename|
+-----------+
|   LUIS POP|
|SHERI GOMES|
+-----------+

+-----------+
|      ename|
+-----------+
|   LUIS POP|
|SHERI GOMES|
+-----------+



## <a id='q12'></a>Q.12 How many customers serviced their two wheelers.

In [44]:
# Q.12 How many customers serviced their two wheelers.
# -- DataFrame API Solution --
count = ser_det_df.where(F.lower(F.col("type_veh")) == "two wheeler").select("cid").distinct().count()
print(f"Number of customers who serviced two-wheelers: {count}")
# -- Spark SQL Solution --
spark.sql("""
    SELECT COUNT(DISTINCT cid) AS two_wheeler_customers
    FROM ser_det_table
    WHERE type_veh = 'TWO WHEELER'
""").show()

Number of customers who serviced two-wheelers: 5
+---------------------+
|two_wheeler_customers|
+---------------------+
|                    5|
+---------------------+



## <a id='q13'></a>Q.13 List the Purchased Items which are used for Customer Service with Unit of that Item.

In [45]:
# Q.13 List the Purchased Items which are used for Customer Service with Unit of that Item.
# -- DataFrame API Solution --
sparepart_df.join(ser_det_df, sparepart_df.spid == ser_det_df.spid, "inner").select("spname", "spunit").distinct().show()
# -- Spark SQL Solution --
spark.sql("""
    SELECT DISTINCT sp.spname, sp.spunit
    FROM sparepart_table sp
    JOIN ser_det_table s ON sp.spid = s.spid
""").show()

+--------------------+------+
|              spname|spunit|
+--------------------+------+
|TWO WHEELER ENGIN...|  LTRS|
|    TWO WHEELER TUBE|   NOS|
|TWO WHEELER CLUTC...|  MTRS|
|        WHITE COLOUR|  LTRS|
+--------------------+------+

+--------------------+------+
|              spname|spunit|
+--------------------+------+
|TWO WHEELER ENGIN...|  LTRS|
|    TWO WHEELER TUBE|   NOS|
|TWO WHEELER CLUTC...|  MTRS|
|        WHITE COLOUR|  LTRS|
+--------------------+------+



## <a id='q14'></a>Q.14 Customers who have Colored their vehicles.

In [46]:
# Q.14 Customers who have Colored their vehicles.
# -- DataFrame API Solution --
customer_df.join(ser_det_df, customer_df.cid == ser_det_df.cid).where(F.lower(F.col("typ_ser")) == "color").select("cname").distinct().show()
# -- Spark SQL Solution --
spark.sql("""
    SELECT DISTINCT c.cname
    FROM customer_table c
    JOIN ser_det_table s ON c.cid = s.cid
    WHERE s.typ_ser = 'COLOR'
""").show()

+------------+
|       cname|
+------------+
|   CHRISTANA|
|KAMILA JOSEF|
+------------+

+------------+
|       cname|
+------------+
|   CHRISTANA|
|KAMILA JOSEF|
+------------+



## <a id='q15'></a>Q.15 Find the annual income of each employee inclusive of Commission.

In [47]:
# Q.15 Find the annual income of each employee inclusive of Commission.
# -- DataFrame API Solution --
commission_agg = ser_det_df.groupBy("eid").agg(F.sum("comm").alias("total_comm"))
employee_df.join(commission_agg, "eid", "left").na.fill(0, ["total_comm"]).withColumn("annual_income", (F.col("esal") * 12) + F.col("total_comm")).select("ename", "annual_income").show()
# -- Spark SQL Solution --
spark.sql("""
    SELECT e.ename, (e.esal * 12) + COALESCE(s.total_comm, 0) AS annual_income
    FROM employee_table e
    LEFT JOIN (SELECT eid, SUM(comm) as total_comm FROM ser_det_table GROUP BY eid) s ON e.eid = s.eid
""").show()

+------------+-------------+
|       ename|annual_income|
+------------+-------------+
| STEVEN KING|        14700|
|DAVID AUSTIN|        13300|
|BRUCE ERENST|        26400|
|    LUIS POP|        20400|
| SHERI GOMES|        12000|
|JAMES PHILIP|         NULL|
+------------+-------------+

+------------+-------------+
|       ename|annual_income|
+------------+-------------+
| STEVEN KING|        14700|
|DAVID AUSTIN|        13300|
|BRUCE ERENST|        26400|
|    LUIS POP|        20400|
| SHERI GOMES|        12000|
|JAMES PHILIP|         NULL|
+------------+-------------+



## <a id='q16'></a>Q.16 Vendor Names who provides the engine oil.

In [48]:
# Q.16 Vendor Names who provides the engine oil.
# -- DataFrame API Solution --
vendor_df.join(purchase_df, "vid").join(sparepart_df, "spid").where(F.col("spname").like("%ENGINE OIL%")).select("vname").distinct().show()
# -- Spark SQL Solution --
spark.sql("""
    SELECT DISTINCT v.vname
    FROM vendor_table v
    JOIN purchase_table p ON v.vid = p.vid
    JOIN sparepart_table sp ON p.spid = sp.spid
    WHERE sp.spname LIKE '%ENGINE OIL%'
""").show()

+------------+
|       vname|
+------------+
| KIRAN PATIL|
|PRAKASH JAIN|
+------------+

+------------+
|       vname|
+------------+
| KIRAN PATIL|
|PRAKASH JAIN|
+------------+



## <a id='q17'></a>Q.17 Total Cost to purchase the Color and name the color purchased.

In [49]:
# Q.17 Total Cost to purchase the Color and name the color purchased.
# -- DataFrame API Solution --
purchase_df.join(sparepart_df, "spid").where(F.col("spname").like("%COLOUR%")).groupBy("spname").agg(F.sum("total").alias("total_purchase_cost")).show()
# -- Spark SQL Solution --
spark.sql("""
    SELECT sp.spname, SUM(p.total) AS total_purchase_cost
    FROM purchase_table p
    JOIN sparepart_table sp ON p.spid = sp.spid
    WHERE sp.spname LIKE '%COLOUR%'
    GROUP BY sp.spname
""").show()

+------------+-------------------+
|      spname|total_purchase_cost|
+------------+-------------------+
|WHITE COLOUR|             1938.0|
+------------+-------------------+

+------------+-------------------+
|      spname|total_purchase_cost|
+------------+-------------------+
|WHITE COLOUR|             1938.0|
+------------+-------------------+



## <a id='q18'></a>Q.18 Purchased Items which are not used in "Ser_det".

In [50]:
# Q.18 Purchased Items which are not used in "Ser_det".
# -- DataFrame API Solution --
purchased_spids = purchase_df.select("spid").distinct()
serviced_spids = ser_det_df.select("spid").distinct()
unused_spids = purchased_spids.subtract(serviced_spids)
unused_spids.join(sparepart_df, "spid").select("spname").show()
# -- Spark SQL Solution --
spark.sql("""
    SELECT DISTINCT sp.spname
    FROM sparepart_table sp
    WHERE sp.spid IN (SELECT spid FROM purchase_table)
      AND sp.spid NOT IN (SELECT spid FROM ser_det_table)
""").show()

+--------------------+
|              spname|
+--------------------+
|FOUR WHEELER ENGI...|
|TWO WHEELER CARBO...|
|TWO WHEELER TAIIL...|
|TWO WHEELER INDIC...|
+--------------------+

+--------------------+
|              spname|
+--------------------+
|TWO WHEELER CARBO...|
|TWO WHEELER INDIC...|
|FOUR WHEELER ENGI...|
|TWO WHEELER TAIIL...|
+--------------------+



## <a id='q19'></a>Q.19 Spare Parts Not Purchased but existing in Sparepart.

In [51]:
# Q.19 Spare Parts Not Purchased but existing in Sparepart.
# -- DataFrame API Solution --
sparepart_df.join(purchase_df, "spid", "left_anti").select("spname").show()
# -- Spark SQL Solution --
spark.sql("""
    SELECT spname FROM sparepart_table
    WHERE spid NOT IN (SELECT spid FROM purchase_table)
""").show()

+--------------------+
|              spname|
+--------------------+
| FOUR WHEELER GASKIT|
|        BLACK COLOUR|
|TWO WHEELER SIDE ...|
|FOUR WHEELER SIDE...|
| TWO WHEELER SHOCKUP|
| FOUR WHEELER BUMPER|
|FOUR WHEELER FRON...|
+--------------------+

+--------------------+
|              spname|
+--------------------+
| FOUR WHEELER GASKIT|
|        BLACK COLOUR|
|TWO WHEELER SIDE ...|
|FOUR WHEELER SIDE...|
| TWO WHEELER SHOCKUP|
| FOUR WHEELER BUMPER|
|FOUR WHEELER FRON...|
+--------------------+



## <a id='q20'></a>Q.20 Calculate the Profit/Loss of the Firm.

In [52]:
# Q.20 Calculate the Profit/Loss of the Firm.
# -- DataFrame API Solution --
total_revenue = ser_det_df.agg(F.sum("total")).collect()[0][0]
total_salary_cost = employee_df.agg(F.sum("esal")).collect()[0][0]
total_purchase_cost = purchase_df.agg(F.sum("total")).collect()[0][0]
profit_loss = total_revenue - (total_salary_cost + total_purchase_cost)
print(f"Firm Profit/Loss: {profit_loss}")
# -- Spark SQL Solution --
spark.sql("""
    SELECT
        (SELECT SUM(total) FROM ser_det_table) -
        ((SELECT SUM(esal) FROM employee_table) + (SELECT SUM(total) FROM purchase_table))
        AS profit_or_loss
""").show()

Firm Profit/Loss: -140645.36000000002
+-------------------+
|     profit_or_loss|
+-------------------+
|-140645.36000000002|
+-------------------+



## <a id='q21'></a>Q.21 Specify the names of customers who have serviced their vehicles more than one time.

In [53]:
# Q.21 Specify the names of customers who have serviced their vehicles more than one time.
# -- DataFrame API Solution --
customer_counts = ser_det_df.groupBy("cid").count().where(F.col("count") > 1)
customer_counts.join(customer_df, "cid").select("cname").show()
# -- Spark SQL Solution --
spark.sql("""
    SELECT c.cname
    FROM customer_table c
    JOIN ser_det_table s ON c.cid = s.cid
    GROUP BY c.cid, c.cname
    HAVING COUNT(s.sid) > 1
""").show()

+----------+
|     cname|
+----------+
|JOHN SMITH|
+----------+

+----------+
|     cname|
+----------+
|JOHN SMITH|
+----------+



## <a id='q22'></a>Q.22 List the Items purchased from vendors locationwise.

In [54]:
# Q.22 List the Items purchased from vendors locationwise.
# -- DataFrame API Solution --
vendor_df.join(purchase_df, "vid").join(sparepart_df, "spid").select(F.col("vadd").alias("location"), F.col("spname").alias("item_name")).orderBy("location").show()
# -- Spark SQL Solution --
spark.sql("""
    SELECT v.vadd AS location, sp.spname AS item_name
    FROM vendor_table v
    JOIN purchase_table p ON v.vid = p.vid
    JOIN sparepart_table sp ON p.spid = sp.spid
    ORDER BY v.vadd
""").show()

+---------+--------------------+
| location|           item_name|
+---------+--------------------+
| BULDHANA|TWO WHEELER TAIIL...|
|   MUNBAI|TWO WHEELER ENGIN...|
|   NASHIK|TWO WHEELER CARBO...|
|OSMANABAD|        WHITE COLOUR|
|OSMANABAD|TWO WHEELER CLUTC...|
|OSMANABAD|TWO WHEELER CLUTC...|
|OSMANABAD|TWO WHEELER INDIC...|
|     PUNE|FOUR WHEELER ENGI...|
|     PUNE|    TWO WHEELER TUBE|
|   SATARA|TWO WHEELER CLUTC...|
+---------+--------------------+

+---------+--------------------+
| location|           item_name|
+---------+--------------------+
| BULDHANA|TWO WHEELER TAIIL...|
|   MUNBAI|TWO WHEELER ENGIN...|
|   NASHIK|TWO WHEELER CARBO...|
|OSMANABAD|        WHITE COLOUR|
|OSMANABAD|TWO WHEELER CLUTC...|
|OSMANABAD|TWO WHEELER CLUTC...|
|OSMANABAD|TWO WHEELER INDIC...|
|     PUNE|FOUR WHEELER ENGI...|
|     PUNE|    TWO WHEELER TUBE|
|   SATARA|TWO WHEELER CLUTC...|
+---------+--------------------+



## <a id='q23'></a>Q.23 Display count of two wheeler and four wheeler from ser_details.

In [55]:
# Q.23 Display count of two wheeler and four wheeler from ser_details.
# -- DataFrame API Solution --
ser_det_df.groupBy("type_veh").count().show()
# -- Spark SQL Solution --
spark.sql("""
    SELECT type_veh, COUNT(*)
    FROM ser_det_table
    GROUP BY type_veh
""").show()

+------------+-----+
|    type_veh|count|
+------------+-----+
| TWO WHEELER|    6|
|FOUR WHEELER|    1|
+------------+-----+

+------------+--------+
|    type_veh|count(1)|
+------------+--------+
| TWO WHEELER|       6|
|FOUR WHEELER|       1|
+------------+--------+



## <a id='q24'></a>Q.24 Display name of customers who paid highest SPGST and for which item.

In [56]:
# Q.24 Display name of customers who paid highest SPGST and for which item.
# -- DataFrame API Solution --
max_spgst = purchase_df.agg(F.max("spgst")).collect()[0][0]
spid_with_max_spgst = purchase_df.where(F.col("spgst") == max_spgst).select("spid")
ser_det_df.join(spid_with_max_spgst, "spid").join(customer_df, "cid").join(sparepart_df, "spid").select("cname", "spname").distinct().show()
# -- Spark SQL Solution --
spark.sql("""
    SELECT DISTINCT c.cname, sp.spname
    FROM ser_det_table s
    JOIN customer_table c ON s.cid = c.cid
    JOIN sparepart_table sp ON s.spid = sp.spid
    WHERE s.spid IN (
        SELECT spid FROM purchase_table WHERE spgst = (SELECT MAX(spgst) FROM purchase_table)
    )
""").show()

+-----+------+
|cname|spname|
+-----+------+
+-----+------+

+-----+------+
|cname|spname|
+-----+------+
+-----+------+



## <a id='q25'></a>Q.25 Display vendors name who have charged highest SPGST rate for which item.

In [57]:
# Q.25 Display vendors name who have charged highest SPGST rate for which item.
# -- DataFrame API Solution --
max_spgst = purchase_df.agg(F.max("spgst")).collect()[0][0]
purchase_df.where(F.col("spgst") == max_spgst).join(vendor_df, "vid").join(sparepart_df, "spid").select("vname", "spname").show()
# -- Spark SQL Solution --
spark.sql("""
    SELECT v.vname, sp.spname
    FROM vendor_table v
    JOIN purchase_table p ON v.vid = p.vid
    JOIN sparepart_table sp ON p.spid = sp.spid
    WHERE p.spgst = (SELECT MAX(spgst) FROM purchase_table)
""").show()

+-----------+--------------------+
|      vname|              spname|
+-----------+--------------------+
|KIRAN PATIL|FOUR WHEELER ENGI...|
+-----------+--------------------+

+-----------+--------------------+
|      vname|              spname|
+-----------+--------------------+
|KIRAN PATIL|FOUR WHEELER ENGI...|
+-----------+--------------------+



## <a id='q26'></a>Q.26 List name of item and employee name who have received item.

In [58]:
# Q.26 List name of item and employee name who have received item.
# -- DataFrame API Solution --
employee_df.join(ser_det_df, "eid").join(sparepart_df, "spid").select("ename", "spname").distinct().show()
# -- Spark SQL Solution --
spark.sql("""
    SELECT DISTINCT e.ename, sp.spname
    FROM employee_table e
    JOIN ser_det_table s ON e.eid = s.eid
    JOIN sparepart_table sp ON s.spid = sp.spid
""").show()

+------------+--------------------+
|       ename|              spname|
+------------+--------------------+
| STEVEN KING|        WHITE COLOUR|
|    LUIS POP|TWO WHEELER CLUTC...|
| STEVEN KING|    TWO WHEELER TUBE|
|DAVID AUSTIN|TWO WHEELER ENGIN...|
+------------+--------------------+

+------------+--------------------+
|       ename|              spname|
+------------+--------------------+
| STEVEN KING|        WHITE COLOUR|
|    LUIS POP|TWO WHEELER CLUTC...|
| STEVEN KING|    TWO WHEELER TUBE|
|DAVID AUSTIN|TWO WHEELER ENGIN...|
+------------+--------------------+



## <a id='q27'></a>Q.27 Display Customer Name, Vehicle Number, Item Used, Purchase Date, Vendor, and Location.

In [59]:
# Q.27 Display Customer Name, Vehicle Number, Item Used, Purchase Date, Vendor, and Location.
# -- DataFrame API Solution --
ser_det_df.join(customer_df, "cid").join(sparepart_df, "spid").join(purchase_df, "spid").join(vendor_df, "vid").select("cname", "veh_no", "spname", "pdate", "vname", "vadd").distinct().show()
# -- Spark SQL Solution --
spark.sql("""
    SELECT DISTINCT c.cname, s.veh_no, sp.spname, p.pdate, v.vname, v.vadd
    FROM ser_det_table s
    JOIN customer_table c ON s.cid = c.cid
    JOIN sparepart_table sp ON s.spid = sp.spid
    JOIN purchase_table p ON s.spid = p.spid
    JOIN vendor_table v ON p.vid = v.vid
""").show()

+------------+----------+--------------------+---------+------------+---------+
|       cname|    veh_no|              spname|    pdate|       vname|     vadd|
+------------+----------+--------------------+---------+------------+---------+
|   CHRISTANA| MH14PA335|        WHITE COLOUR|20-OCT-11|RAM KULKARNI|OSMANABAD|
|  JOHN SMITH| MH16U5713|TWO WHEELER ENGIN...|02-MAR-11|PRAKASH JAIN|   MUNBAI|
| JORDEN WOOD|MH12PQ1313|TWO WHEELER CLUTC...|15-OCT-11|RAM KULKARNI|OSMANABAD|
| JORDEN WOOD|MH12PQ1313|TWO WHEELER CLUTC...|12-OCT-11|RAM KULKARNI|OSMANABAD|
|KAMILA JOSEF| MH12WE334|        WHITE COLOUR|20-OCT-11|RAM KULKARNI|OSMANABAD|
| JORDEN WOOD|MH12PQ1313|TWO WHEELER CLUTC...|22-AUG-11| AMOL SHENDE|   SATARA|
| cyona blake|MH15CA3228|    TWO WHEELER TUBE|01-JAN-11| KIRAN PATIL|     PUNE|
| ANDRU SYMON|MH17BB1345|    TWO WHEELER TUBE|01-JAN-11| KIRAN PATIL|     PUNE|
+------------+----------+--------------------+---------+------------+---------+

+------------+----------+--------------

## <a id='q28'></a>Q.28 who belong this vehicle 'MH14PA335'?

In [60]:
# Q.28 who belong this vehicle 'MH14PA335'?
# -- DataFrame API Solution --
ser_det_df.where(F.col("veh_no") == 'MH14PA335').join(customer_df, "cid").select("cname").show()
# -- Spark SQL Solution --
spark.sql("""
    SELECT c.cname
    FROM customer_table c
    JOIN ser_det_table s ON c.cid = s.cid
    WHERE s.veh_no = 'MH14PA335'
""").show()

+---------+
|    cname|
+---------+
|CHRISTANA|
+---------+

+---------+
|    cname|
+---------+
|CHRISTANA|
+---------+



## <a id='q29'></a>Q.29 Display the name of customer from New York and their vehicle service date.

In [61]:
# Q.29 Display the name of customer from New York and their vehicle service date.
# -- DataFrame API Solution --
customer_df.where(F.col("cadd") == 'NEW YORK').join(ser_det_df, "cid").select("cname", "ser_date").show()
# -- Spark SQL Solution --
spark.sql("""
    SELECT c.cname, s.ser_date
    FROM customer_table c
    JOIN ser_det_table s ON c.cid = s.cid
    WHERE c.cadd = 'NEW YORK'
""").show()

+-----------+---------+
|      cname| ser_date|
+-----------+---------+
|cyona blake|02-JAN-11|
+-----------+---------+

+-----------+---------+
|      cname| ser_date|
+-----------+---------+
|cyona blake|02-JAN-11|
+-----------+---------+



## <a id='q30'></a>Q.30 from whom we have purchased items having maximum cost?

In [62]:
# Q.30 from whom we have purchased items having maximum cost?
# -- DataFrame API Solution --
max_purchase_total = purchase_df.agg(F.max("total")).collect()[0][0]
purchase_df.where(F.col("total") == max_purchase_total).join(vendor_df, "vid").select("vname").show()
# -- Spark SQL Solution --
spark.sql("""
    SELECT v.vname
    FROM vendor_table v
    JOIN purchase_table p ON v.vid = p.vid
    WHERE p.total = (SELECT MAX(total) FROM purchase_table)
""").show()

+-----------+
|      vname|
+-----------+
|KIRAN PATIL|
+-----------+

+-----------+
|      vname|
+-----------+
|KIRAN PATIL|
+-----------+



## <a id='q31'></a>Q.31 Display employees who are not 'Mechanic' and have done services.

In [63]:
# Q.31 Display employees who are not 'Mechanic' and have done services.
# -- DataFrame API Solution --
employee_df.where(F.col("ejob") != 'MECHANIC').join(ser_det_df, "eid", "inner").select("ename").distinct().show()
# -- Spark SQL Solution --
spark.sql("""
    SELECT DISTINCT e.ename
    FROM employee_table e
    JOIN ser_det_table s ON e.eid = s.eid
    WHERE e.ejob != 'MECHANIC'
""").show()

+------------+
|       ename|
+------------+
| STEVEN KING|
|DAVID AUSTIN|
+------------+

+------------+
|       ename|
+------------+
| STEVEN KING|
|DAVID AUSTIN|
+------------+



## <a id='q32'></a>Q.32 Display jobs with more than two employees.

In [64]:
# Q.32 Display jobs with more than two employees.
# -- DataFrame API Solution --
employee_df.groupBy("ejob").count().where(F.col("count") > 1).show()  # > 1 for sample data
# -- Spark SQL Solution --
spark.sql("""
    SELECT ejob, COUNT(*)
    FROM employee_table
    GROUP BY ejob
    HAVING COUNT(*) > 1
""").show()

+--------+-----+
|    ejob|count|
+--------+-----+
|MECHANIC|    2|
|  FITTER|    3|
+--------+-----+

+--------+--------+
|    ejob|count(1)|
+--------+--------+
|MECHANIC|       2|
|  FITTER|       3|
+--------+--------+



## <a id='q33'></a>Q.33 Display details of employees who did a service and rank them by number of services.

In [65]:
# Q.33 Display details of employees who did a service and rank them by number of services.
# -- DataFrame API Solution --
service_counts = ser_det_df.groupBy("eid").count()
window_spec = Window.orderBy(F.col("count").desc())
employee_df.join(service_counts, "eid").select("ename", "ejob", "count").withColumn("service_rank", F.rank().over(window_spec)).show()
# -- Spark SQL Solution --
spark.sql("""
    SELECT e.ename, e.ejob, COUNT(s.sid) AS service_count,
           RANK() OVER (ORDER BY COUNT(s.sid) DESC) AS service_rank
    FROM employee_table e
    JOIN ser_det_table s ON e.eid = s.eid
    GROUP BY e.eid, e.ename, e.ejob
""").show()

+------------+--------+-----+------------+
|       ename|    ejob|count|service_rank|
+------------+--------+-----+------------+
| STEVEN KING| PAINTER|    4|           1|
|DAVID AUSTIN|  FITTER|    2|           2|
|    LUIS POP|MECHANIC|    1|           3|
+------------+--------+-----+------------+

+------------+--------+-------------+------------+
|       ename|    ejob|service_count|service_rank|
+------------+--------+-------------+------------+
| STEVEN KING| PAINTER|            4|           1|
|DAVID AUSTIN|  FITTER|            2|           2|
|    LUIS POP|MECHANIC|            1|           3|
+------------+--------+-------------+------------+



## <a id='q34'></a>Q.34 Display painter/fitter employees who provided a service and total service count for each.

In [66]:
# Q.34 Display painter/fitter employees who provided a service and total service count for each.
# -- DataFrame API Solution --
employee_df.where(F.col("ejob").isin(['PAINTER', 'FITTER'])).join(ser_det_df, "eid", "inner").groupBy("ename", "ejob").count().show()
# -- Spark SQL Solution --
spark.sql("""
    SELECT e.ename, e.ejob, COUNT(s.sid) AS service_count
    FROM employee_table e
    JOIN ser_det_table s ON e.eid = s.eid
    WHERE e.ejob IN ('PAINTER', 'FITTER')
    GROUP BY e.eid, e.ename, e.ejob
""").show()

+------------+-------+-----+
|       ename|   ejob|count|
+------------+-------+-----+
| STEVEN KING|PAINTER|    4|
|DAVID AUSTIN| FITTER|    2|
+------------+-------+-----+

+------------+-------+-------------+
|       ename|   ejob|service_count|
+------------+-------+-------------+
| STEVEN KING|PAINTER|            4|
|DAVID AUSTIN| FITTER|            2|
+------------+-------+-------------+



## <a id='q35'></a>Q.35 Display employee salary and provide a Grade based on salary.

In [69]:
# Q.35 Display employee salary and provide a Grade based on salary.
# -- DataFrame API Solution --
window_spec = Window.orderBy(F.col("esal").desc())
employee_df.withColumn("rank", F.dense_rank().over(window_spec)).withColumn("salary_grade", F.when(F.col("rank") <= 2, 'A').when(F.col("rank") <= 4, 'B').otherwise('C')).select("ename", "esal", "salary_grade").show()
# -- Spark SQL Solution --
spark.sql("""
    SELECT ename, esal,
        CASE
            WHEN DENSE_RANK() OVER (ORDER BY esal DESC NULLS LAST) <= 2 THEN 'A'
            WHEN DENSE_RANK() OVER (ORDER BY esal DESC NULLS LAST) <= 4 THEN 'B'
            ELSE 'C'
        END AS salary_grade
    FROM employee_table
""").show()

+------------+----+------------+
|       ename|esal|salary_grade|
+------------+----+------------+
|BRUCE ERENST|2200|           A|
|    LUIS POP|1700|           A|
| STEVEN KING|1200|           B|
|DAVID AUSTIN|1100|           B|
| SHERI GOMES|1000|           C|
|JAMES PHILIP|NULL|           C|
+------------+----+------------+

+------------+----+------------+
|       ename|esal|salary_grade|
+------------+----+------------+
|BRUCE ERENST|2200|           A|
|    LUIS POP|1700|           A|
| STEVEN KING|1200|           B|
|DAVID AUSTIN|1100|           B|
| SHERI GOMES|1000|           C|
|JAMES PHILIP|NULL|           C|
+------------+----+------------+



## <a id='q36'></a>Q.36 display the 4th record of emp table without using group by and rowid.

In [70]:
# Q.36 display the 4th record of emp table without using group by and rowid.
# -- DataFrame API Solution --
window_spec = Window.orderBy("eid")
employee_df.withColumn("rn", F.row_number().over(window_spec)).where(F.col("rn") == 4).drop("rn").show()
# -- Spark SQL Solution --
spark.sql("SELECT * FROM (SELECT *, ROW_NUMBER() OVER (ORDER BY eid) as rn FROM employee_table) WHERE rn = 4").show()

+----+------------+--------+---------+--------+----+---------+----+
| eid|       ename|    ejob|     eadd|econtact|esal|     edoj|edol|
+----+------------+--------+---------+--------+----+---------+----+
|3003|BRUCE ERENST|MECHANIC|NEW JERCY|10367264|2200|08-SEP-10|NULL|
+----+------------+--------+---------+--------+----+---------+----+

+----+------------+--------+---------+--------+----+---------+----+---+
| eid|       ename|    ejob|     eadd|econtact|esal|     edoj|edol| rn|
+----+------------+--------+---------+--------+----+---------+----+---+
|3003|BRUCE ERENST|MECHANIC|NEW JERCY|10367264|2200|08-SEP-10|NULL|  4|
+----+------------+--------+---------+--------+----+---------+----+---+



## <a id='q37'></a>Q.37 Provide a commission 100 to employees who are not earning any commission.

In [71]:
# Q.37 Provide a commission 100 to employees who are not earning any commission.
# -- DataFrame API Solution --
ser_det_df.withColumn("new_commission", F.when(F.col("comm") == 0, 100).otherwise(F.col("comm"))).select("sid", "eid", "comm", "new_commission").show()
# -- Spark SQL Solution --
spark.sql("""
    SELECT sid, eid, comm,
           CASE WHEN comm = 0 THEN 100 ELSE comm END AS new_commission
    FROM ser_det_table
""").show()

+----+----+----+--------------+
| sid| eid|comm|new_commission|
+----+----+----+--------------+
|6001|3001|   0|           100|
|6002|3002|  50|            50|
|6003|3004|   0|           100|
|6004|3002|  50|            50|
|6005|3001| 150|           150|
|6006|3001| 150|           150|
|6007|3001|   0|           100|
+----+----+----+--------------+

+----+----+----+--------------+
| sid| eid|comm|new_commission|
+----+----+----+--------------+
|6001|3001|   0|           100|
|6002|3002|  50|            50|
|6003|3004|   0|           100|
|6004|3002|  50|            50|
|6005|3001| 150|           150|
|6006|3001| 150|           150|
|6007|3001|   0|           100|
+----+----+----+--------------+



## <a id='q38'></a>Q.38 totals no. of services for each day and place the results in descending order.

In [72]:
# Q.38 totals no. of services for each day and place the results in descending order.
# -- DataFrame API Solution --
ser_det_df.groupBy("ser_date").count().orderBy(F.col("count").desc()).show()
# -- Spark SQL Solution --
spark.sql("""
    SELECT ser_date, COUNT(sid) AS number_of_services
    FROM ser_det_table
    GROUP BY ser_date
    ORDER BY number_of_services DESC
""").show()

+---------+-----+
| ser_date|count|
+---------+-----+
|05-MAY-11|    1|
|22-AUG-11|    1|
|02-JAN-11|    1|
|21-OCT-11|    1|
|04-MAR-11|    1|
|01-JAN-12|    1|
|01-DEC-11|    1|
+---------+-----+

+---------+------------------+
| ser_date|number_of_services|
+---------+------------------+
|05-MAY-11|                 1|
|22-AUG-11|                 1|
|02-JAN-11|                 1|
|21-OCT-11|                 1|
|04-MAR-11|                 1|
|01-JAN-12|                 1|
|01-DEC-11|                 1|
+---------+------------------+



## <a id='q39'></a>Q.39 Display the service details of those customer who belong from same city.

In [73]:
# Q.39 Display the service details of those customer who belong from same city.
# -- DataFrame API Solution --
city_counts = customer_df.groupBy("cadd").count().where(F.col("count") > 1)
multi_cust_cities = city_counts.select("cadd")
customer_df.join(multi_cust_cities, "cadd").join(ser_det_df, "cid").select(ser_det_df["*"]).show()
# -- Spark SQL Solution --
spark.sql("""
    SELECT s.*
    FROM ser_det_table s
    JOIN customer_table c1 ON s.cid = c1.cid
    WHERE c1.cadd IN (
        SELECT cadd FROM customer_table GROUP BY cadd HAVING COUNT(*) > 1
    )
""").show()

+----+----+----+----+-----------+----------+------------+---------+---+-------+------+----+-------+----+-----+
| sid| cid| eid|spid|   type_veh|    veh_no|     typ_ser| ser_date|qty|sp_rate|sp_amt|sp_g|ser_amt|comm|total|
+----+----+----+----+-----------+----------+------------+---------+---+-------+------+----+-------+----+-----+
|6001|1001|3001|4001|TWO WHEELER|MH15CA3228|TUBE DAMAGED|02-JAN-11|1.0|    250|   250|  35|     50|   0|  335|
+----+----+----+----+-----------+----------+------------+---------+---+-------+------+----+-------+----+-----+

+----+----+----+----+-----------+----------+------------+---------+---+-------+------+----+-------+----+-----+
| sid| cid| eid|spid|   type_veh|    veh_no|     typ_ser| ser_date|qty|sp_rate|sp_amt|sp_g|ser_amt|comm|total|
+----+----+----+----+-----------+----------+------------+---------+---+-------+------+----+-------+----+-----+
|6001|1001|3001|4001|TWO WHEELER|MH15CA3228|TUBE DAMAGED|02-JAN-11|1.0|    250|   250|  35|     50|   0|  335|


## <a id='q40'></a>Q.40 find all pairs of customers serviced by a single employee.

In [89]:
# Q.40 find all pairs of customers serviced by a single employee.

# Rename columns before join to avoid ambiguity
s1 = ser_det_df.selectExpr("cid as cid1", "eid as eid1").alias("s1")
s2 = ser_det_df.selectExpr("cid as cid2", "eid as eid2").alias("s2")

c1 = customer_df.selectExpr("cid as cid1", "cname as cname1").alias("c1")
c2 = customer_df.selectExpr("cid as cid2", "cname as cname2").alias("c2")

# Join on employee id and ensure cid1 < cid2 (to avoid duplicate/reverse pairs)
employee_service_pairs = (
    s1.join(s2, (s1["eid1"] == s2["eid2"]) & (s1["cid1"] < s2["cid2"]))
      .join(c1, "cid1")
      .join(c2, "cid2")
)

result_df = employee_service_pairs.select(
    "cname1", "cname2", "eid1"
).distinct()

result_df.show()


spark.sql("""
    SELECT DISTINCT c1.cname AS customer1,
                    c2.cname AS customer2,
                    s1.eid     AS employee_id
    FROM ser_det_table s1
    JOIN ser_det_table s2
      ON s1.eid = s2.eid AND s1.cid < s2.cid
    JOIN customer_table c1
      ON s1.cid = c1.cid
    JOIN customer_table c2
      ON s2.cid = c2.cid
""").show()


+------------+------------+----+
|      cname1|      cname2|eid1|
+------------+------------+----+
|KAMILA JOSEF| ANDRU SYMON|3001|
| cyona blake| ANDRU SYMON|3001|
| cyona blake|KAMILA JOSEF|3001|
|   CHRISTANA| ANDRU SYMON|3001|
| cyona blake|   CHRISTANA|3001|
|   CHRISTANA|KAMILA JOSEF|3001|
+------------+------------+----+

+------------+------------+-----------+
|   customer1|   customer2|employee_id|
+------------+------------+-----------+
|KAMILA JOSEF| ANDRU SYMON|       3001|
| cyona blake| ANDRU SYMON|       3001|
| cyona blake|KAMILA JOSEF|       3001|
|   CHRISTANA| ANDRU SYMON|       3001|
| cyona blake|   CHRISTANA|       3001|
|   CHRISTANA|KAMILA JOSEF|       3001|
+------------+------------+-----------+



## <a id='q41'></a>Q.41 List each service number follow by name of the customer who made that service.

In [90]:
# Q.41 List each service number follow by name of the customer who made that service.
# -- DataFrame API Solution --
ser_det_df.join(customer_df, "cid").select("sid", "cname").show()
# -- Spark SQL Solution --
spark.sql("""
    SELECT s.sid, c.cname
    FROM ser_det_table s
    JOIN customer_table c ON s.cid = c.cid
""").show()

+----+------------+
| sid|       cname|
+----+------------+
|6001| cyona blake|
|6004|  JOHN SMITH|
|6002|  JOHN SMITH|
|6003| JORDEN WOOD|
|6005|   CHRISTANA|
|6006|KAMILA JOSEF|
|6007| ANDRU SYMON|
+----+------------+

+----+------------+
| sid|       cname|
+----+------------+
|6001| cyona blake|
|6004|  JOHN SMITH|
|6002|  JOHN SMITH|
|6003| JORDEN WOOD|
|6005|   CHRISTANA|
|6006|KAMILA JOSEF|
|6007| ANDRU SYMON|
+----+------------+



## <a id='q42'></a>Q.42 Provide employee rating (A,B,C,D) based on number of services.

In [91]:
# Q.42 Provide employee rating (A,B,C,D) based on number of services.
# -- DataFrame API Solution --
service_counts_df = ser_det_df.groupBy("eid").count()
employee_df.join(service_counts_df, "eid", "left").na.fill(0).withColumn("rating", F.when(F.col("count") >= 3, 'A').when(F.col("count") == 2, 'B').when(F.col("count") == 1, 'C').otherwise('D')).select("ename", "ejob", "count", "rating").show()
# -- Spark SQL Solution --
spark.sql("""
    SELECT e.ename, e.ejob, sc.service_count,
        CASE
            WHEN sc.service_count >= 3 THEN 'A'
            WHEN sc.service_count = 2 THEN 'B'
            WHEN sc.service_count = 1 THEN 'C'
            ELSE 'D'
        END as rating
    FROM employee_table e
    LEFT JOIN (SELECT eid, COUNT(*) as service_count FROM ser_det_table GROUP BY eid) sc ON e.eid = sc.eid
""").show()

+------------+--------+-----+------+
|       ename|    ejob|count|rating|
+------------+--------+-----+------+
| STEVEN KING| PAINTER|    4|     A|
|DAVID AUSTIN|  FITTER|    2|     B|
|BRUCE ERENST|MECHANIC|    0|     D|
|    LUIS POP|MECHANIC|    1|     C|
| SHERI GOMES|  FITTER|    0|     D|
|JAMES PHILIP|  FITTER|    0|     D|
+------------+--------+-----+------+

+------------+--------+-------------+------+
|       ename|    ejob|service_count|rating|
+------------+--------+-------------+------+
| STEVEN KING| PAINTER|            4|     A|
|DAVID AUSTIN|  FITTER|            2|     B|
|BRUCE ERENST|MECHANIC|         NULL|     D|
|    LUIS POP|MECHANIC|            1|     C|
| SHERI GOMES|  FITTER|         NULL|     D|
|JAMES PHILIP|  FITTER|         NULL|     D|
+------------+--------+-------------+------+



## <a id='q43'></a>Q.43 Get maximum service amount of each customer with their customer details.

In [92]:
# Q.43 Get maximum service amount of each customer with their customer details.
# -- DataFrame API Solution --
max_service_amt_df = ser_det_df.groupBy("cid").agg(F.max("total").alias("max_service_amount"))
customer_df.join(max_service_amt_df, "cid").show()
# -- Spark SQL Solution --
spark.sql("""
    SELECT c.*, MAX(s.total) as max_service_amount
    FROM customer_table c
    JOIN ser_det_table s ON c.cid = s.cid
    GROUP BY c.cid, c.cname, c.cadd, c.contact, c.creditdays, c.date, c.gender
""").show()

+----+------------+---------+--------+----------+---------+------+------------------+
| cid|       cname|     cadd| contact|creditdays|     date|gender|max_service_amount|
+----+------------+---------+--------+----------+---------+------+------------------+
|1001| cyona blake| NEW YORK|12345684|        20|20-JAN-11|female|               335|
|1002|  JOHN SMITH|NEW JERSI| 1341684|        20|21-FEB-11|  male|               752|
|1003| JORDEN WOOD|     PRAG| 1805184|        20|22-MAR-11|  male|               139|
|1004|   CHRISTANA|MANHATTON| 1125684|        31|23-APR-13|female|              1469|
|1006|KAMILA JOSEF|   PRAISE|  124568|         9|28-JUL-11|female|              1469|
|1007| ANDRU SYMON|    TEXAS|  125654|        15|01-APR-16|  male|               435|
+----+------------+---------+--------+----------+---------+------+------------------+

+----+------------+---------+--------+----------+---------+------+------------------+
| cid|       cname|     cadd| contact|creditdays|    

## <a id='q44'></a>Q.44 Get the details of customers with his total no of services.

In [93]:
# Q.44 Get the details of customers with his total no of services.
# -- DataFrame API Solution --
total_services_df = ser_det_df.groupBy("cid").count().withColumnRenamed("count", "total_services")
customer_df.join(total_services_df, "cid", "left").na.fill(0).show()
# -- Spark SQL Solution --
spark.sql("""
    SELECT c.*, COUNT(s.sid) AS total_services
    FROM customer_table c
    LEFT JOIN ser_det_table s ON c.cid = s.cid
    GROUP BY c.cid, c.cname, c.cadd, c.contact, c.creditdays, c.date, c.gender
""").show()

+----+-------------+---------+--------+----------+---------+------+--------------+
| cid|        cname|     cadd| contact|creditdays|     date|gender|total_services|
+----+-------------+---------+--------+----------+---------+------+--------------+
|1001|  cyona blake| NEW YORK|12345684|        20|20-JAN-11|female|             1|
|1002|   JOHN SMITH|NEW JERSI| 1341684|        20|21-FEB-11|  male|             2|
|1003|  JORDEN WOOD|     PRAG| 1805184|        20|22-MAR-11|  male|             1|
|1004|    CHRISTANA|MANHATTON| 1125684|        31|23-APR-13|female|             1|
|1005|     TOM HILL|   LONDON| 1239284|        10|25-JUN-15|  male|             0|
|1006| KAMILA JOSEF|   PRAISE|  124568|         9|28-JUL-11|female|             1|
|1007|  ANDRU SYMON|    TEXAS|  125654|        15|01-APR-16|  male|             1|
|1008|SANJU SAMSUNG| NEW YORK|12346341|         4|20-JAN-16|  male|             0|
+----+-------------+---------+--------+----------+---------+------+--------------+

+--

## <a id='q45'></a>Q.45 From which location sparepart purchased with highest cost?

In [94]:
# Q.45 From which location sparepart purchased with highest cost?
# -- DataFrame API Solution --
max_sprate = purchase_df.agg(F.max("sprate")).collect()[0][0]
purchase_df.where(F.col("sprate") == max_sprate).join(vendor_df, "vid").select("vadd").show()
# -- Spark SQL Solution --
spark.sql("""
    SELECT v.vadd
    FROM vendor_table v
    JOIN purchase_table p ON v.vid = p.vid
    WHERE p.sprate = (SELECT MAX(sprate) FROM purchase_table)
""").show()

+----+
|vadd|
+----+
|PUNE|
+----+

+----+
|vadd|
+----+
|PUNE|
+----+



## <a id='q46'></a>Q.46 Get the details of employee with their service details who has salary is null.

In [95]:
# Q.46 Get the details of employee with their service details who has salary is null.
# -- DataFrame API Solution --
employee_df.where(F.col("esal").isNull()).join(ser_det_df, "eid").select(employee_df.ename, employee_df.ejob, ser_det_df["*"]).show()
# -- Spark SQL Solution --
spark.sql("""
    SELECT e.ename, e.ejob, s.*
    FROM employee_table e
    JOIN ser_det_table s ON e.eid = s.eid
    WHERE e.esal IS NULL
""").show()

+-----+----+---+---+---+----+--------+------+-------+--------+---+-------+------+----+-------+----+-----+
|ename|ejob|sid|cid|eid|spid|type_veh|veh_no|typ_ser|ser_date|qty|sp_rate|sp_amt|sp_g|ser_amt|comm|total|
+-----+----+---+---+---+----+--------+------+-------+--------+---+-------+------+----+-------+----+-----+
+-----+----+---+---+---+----+--------+------+-------+--------+---+-------+------+----+-------+----+-----+

+-----+----+---+---+---+----+--------+------+-------+--------+---+-------+------+----+-------+----+-----+
|ename|ejob|sid|cid|eid|spid|type_veh|veh_no|typ_ser|ser_date|qty|sp_rate|sp_amt|sp_g|ser_amt|comm|total|
+-----+----+---+---+---+----+--------+------+-------+--------+---+-------+------+----+-------+----+-----+
+-----+----+---+---+---+----+--------+------+-------+--------+---+-------+------+----+-------+----+-----+



## <a id='q47'></a>Q.47 find the sum of purchase location wise.

In [96]:
# Q.47 find the sum of purchase location wise.
# -- DataFrame API Solution --
vendor_df.join(purchase_df, "vid").groupBy("vadd").agg(F.sum("total").alias("total_purchase_amount")).show()
# -- Spark SQL Solution --
spark.sql("""
    SELECT v.vadd, SUM(p.total) AS total_purchase_amount
    FROM vendor_table v
    JOIN purchase_table p ON v.vid = p.vid
    GROUP BY v.vadd
""").show()

+---------+---------------------+
|     vadd|total_purchase_amount|
+---------+---------------------+
| BULDHANA|               2320.0|
|   MUNBAI|               1988.0|
|   NASHIK|               6251.6|
|OSMANABAD|    7516.959999999999|
|     PUNE|             119150.0|
|   SATARA|               1569.8|
+---------+---------------------+

+---------+---------------------+
|     vadd|total_purchase_amount|
+---------+---------------------+
| BULDHANA|               2320.0|
|   MUNBAI|               1988.0|
|   NASHIK|               6251.6|
|OSMANABAD|    7516.959999999999|
|     PUNE|             119150.0|
|   SATARA|               1569.8|
+---------+---------------------+



## <a id='q48'></a>Q.48 write a query sum of purchase amount in word location wise?

In [97]:
# Q.48 write a query sum of purchase amount in word location wise?
# -- DataFrame API Solution --
def number_to_words_py(n):
    if n is None:
        return None
    return num2words(int(n))
number_to_words_udf = F.udf(number_to_words_py, StringType())
purchase_sum_df = vendor_df.join(purchase_df, "vid").groupBy("vadd").agg(F.sum("total").alias("total_amount"))
purchase_sum_df.withColumn("amount_in_words", number_to_words_udf(F.col("total_amount"))).show(truncate=False)
# -- Spark SQL Solution --
spark.udf.register("to_words", number_to_words_py, StringType())
spark.sql("""
    SELECT vadd, total_amount, to_words(total_amount) as amount_in_words FROM (
        SELECT v.vadd, SUM(p.total) AS total_amount
        FROM vendor_table v
        JOIN purchase_table p ON v.vid = p.vid
        GROUP BY v.vadd
    )
""").show(truncate=False)

+---------+-----------------+--------------------------------------------------------+
|vadd     |total_amount     |amount_in_words                                         |
+---------+-----------------+--------------------------------------------------------+
|BULDHANA |2320.0           |two thousand, three hundred and twenty                  |
|MUNBAI   |1988.0           |one thousand, nine hundred and eighty-eight             |
|NASHIK   |6251.6           |six thousand, two hundred and fifty-one                 |
|OSMANABAD|7516.959999999999|seven thousand, five hundred and sixteen                |
|PUNE     |119150.0         |one hundred and nineteen thousand, one hundred and fifty|
|SATARA   |1569.8           |one thousand, five hundred and sixty-nine               |
+---------+-----------------+--------------------------------------------------------+

+---------+-----------------+--------------------------------------------------------+
|vadd     |total_amount     |amount_in_wor

## <a id='q49'></a>Q.49 Has the customer who has spent the largest amount money been given the highest rating.

In [98]:
# Q.49 Has the customer who has spent the largest amount money been given the highest rating.
# -- DataFrame API Solution --
spending_df = ser_det_df.groupBy("cid").agg(F.sum("total").alias("total_spent"))
window_spec = Window.orderBy(F.col("total_spent").desc())
rating_df = spending_df.withColumn("rank", F.dense_rank().over(window_spec)).withColumn("rating", F.when(F.col("rank") == 1, 'A').when(F.col("rank") <= 3, 'B').otherwise('C'))
customer_df.join(rating_df, "cid").orderBy(F.col("total_spent").desc()).show()
# -- Spark SQL Solution --
spark.sql("""
    WITH CustomerSpending AS (
        SELECT cid, SUM(total) as total_spent
        FROM ser_det_table
        GROUP BY cid
    ),
    CustomerRating AS (
        SELECT cid, total_spent,
            CASE
                WHEN DENSE_RANK() OVER (ORDER BY total_spent DESC) = 1 THEN 'A'
                WHEN DENSE_RANK() OVER (ORDER BY total_spent DESC) <= 3 THEN 'B'
                ELSE 'C'
            END as rating
        FROM CustomerSpending
    )
    SELECT c.cname, cr.total_spent, cr.rating
    FROM CustomerRating cr
    JOIN customer_table c ON cr.cid = c.cid
    ORDER BY cr.total_spent DESC
""").show()

+----+------------+---------+--------+----------+---------+------+-----------+----+------+
| cid|       cname|     cadd| contact|creditdays|     date|gender|total_spent|rank|rating|
+----+------------+---------+--------+----------+---------+------+-----------+----+------+
|1002|  JOHN SMITH|NEW JERSI| 1341684|        20|21-FEB-11|  male|       1504|   1|     A|
|1004|   CHRISTANA|MANHATTON| 1125684|        31|23-APR-13|female|       1469|   2|     B|
|1006|KAMILA JOSEF|   PRAISE|  124568|         9|28-JUL-11|female|       1469|   2|     B|
|1007| ANDRU SYMON|    TEXAS|  125654|        15|01-APR-16|  male|        435|   3|     B|
|1001| cyona blake| NEW YORK|12345684|        20|20-JAN-11|female|        335|   4|     C|
|1003| JORDEN WOOD|     PRAG| 1805184|        20|22-MAR-11|  male|        139|   5|     C|
+----+------------+---------+--------+----------+---------+------+-----------+----+------+

+------------+-----------+------+
|       cname|total_spent|rating|
+------------+-------

## <a id='q50'></a>Q.50 select the total amount in service for each customer for which the total is greater than the amount of the largest service amount in the table.

In [99]:
# Q.50 select the total amount in service for each customer for which the total is greater than the amount of the largest service amount in the table.
# -- DataFrame API Solution --
largest_service_amount = ser_det_df.agg(F.max("total")).first()[0]
customer_total_spend_df = ser_det_df.groupBy("cid").agg(F.sum("total").alias("customer_total_spend"))
customer_total_spend_df.where(F.col("customer_total_spend") > largest_service_amount).join(customer_df, "cid").select("cname", "customer_total_spend").show()
# -- Spark SQL Solution --
spark.sql("""
    SELECT c.cname, SUM(s.total) as customer_total_service_amount
    FROM customer_table c
    JOIN ser_det_table s ON c.cid = s.cid
    GROUP BY c.cname
    HAVING SUM(s.total) > (SELECT MAX(total) FROM ser_det_table)
""").show()

+----------+--------------------+
|     cname|customer_total_spend|
+----------+--------------------+
|JOHN SMITH|                1504|
+----------+--------------------+

+----------+-----------------------------+
|     cname|customer_total_service_amount|
+----------+-----------------------------+
|JOHN SMITH|                         1504|
+----------+-----------------------------+



## <a id='q51'></a>Q.51 List the customer name and sparepart name used for their vehicle and vehicle type.

In [100]:
# Q.51 List the customer name and sparepart name used for their vehicle and vehicle type.
# -- DataFrame API Solution --
ser_det_df.join(customer_df, "cid").join(sparepart_df, "spid").select("cname", "spname", "type_veh").show()
# -- Spark SQL Solution --
spark.sql("""
    SELECT c.cname, sp.spname, s.type_veh
    FROM ser_det_table s
    JOIN customer_table c ON s.cid = c.cid
    JOIN sparepart_table sp ON s.spid = sp.spid
""").show()

+------------+--------------------+------------+
|       cname|              spname|    type_veh|
+------------+--------------------+------------+
| cyona blake|    TWO WHEELER TUBE| TWO WHEELER|
|  JOHN SMITH|TWO WHEELER ENGIN...| TWO WHEELER|
| JORDEN WOOD|TWO WHEELER CLUTC...| TWO WHEELER|
|  JOHN SMITH|TWO WHEELER ENGIN...| TWO WHEELER|
|   CHRISTANA|        WHITE COLOUR| TWO WHEELER|
|KAMILA JOSEF|        WHITE COLOUR| TWO WHEELER|
| ANDRU SYMON|    TWO WHEELER TUBE|FOUR WHEELER|
+------------+--------------------+------------+

+------------+--------------------+------------+
|       cname|              spname|    type_veh|
+------------+--------------------+------------+
| cyona blake|    TWO WHEELER TUBE| TWO WHEELER|
|  JOHN SMITH|TWO WHEELER ENGIN...| TWO WHEELER|
| JORDEN WOOD|TWO WHEELER CLUTC...| TWO WHEELER|
|  JOHN SMITH|TWO WHEELER ENGIN...| TWO WHEELER|
|   CHRISTANA|        WHITE COLOUR| TWO WHEELER|
|KAMILA JOSEF|        WHITE COLOUR| TWO WHEELER|
| ANDRU SYMON|    T

## <a id='q52'></a>Q.52 Get spname, ename, cname, quantity, rate, service amount for records in service table.

In [101]:
# Q.52 Get spname, ename, cname, quantity, rate, service amount for records in service table.
# -- DataFrame API Solution --
ser_det_df.join(sparepart_df, "spid").join(employee_df, "eid").join(customer_df, "cid").select("spname", "ename", "cname", "qty", "sp_rate", "ser_amt").show()
# -- Spark SQL Solution --
spark.sql("""
    SELECT sp.spname, e.ename, c.cname, s.qty, s.sp_rate, s.ser_amt
    FROM ser_det_table s
    JOIN sparepart_table sp ON s.spid = sp.spid
    JOIN employee_table e ON s.eid = e.eid
    JOIN customer_table c ON s.cid = c.cid
""").show()

+--------------------+------------+------------+---+-------+-------+
|              spname|       ename|       cname|qty|sp_rate|ser_amt|
+--------------------+------------+------------+---+-------+-------+
|    TWO WHEELER TUBE| STEVEN KING| ANDRU SYMON|1.0|    250|    150|
|    TWO WHEELER TUBE| STEVEN KING| cyona blake|1.0|    250|     50|
|TWO WHEELER ENGIN...|DAVID AUSTIN|  JOHN SMITH|1.0|    400|    300|
|TWO WHEELER ENGIN...|DAVID AUSTIN|  JOHN SMITH|1.0|    400|    300|
|TWO WHEELER CLUTC...|    LUIS POP| JORDEN WOOD|1.0|    129|     10|
|        WHITE COLOUR| STEVEN KING|KAMILA JOSEF|2.5|    340|    500|
|        WHITE COLOUR| STEVEN KING|   CHRISTANA|2.5|    340|    500|
+--------------------+------------+------------+---+-------+-------+

+--------------------+------------+------------+---+-------+-------+
|              spname|       ename|       cname|qty|sp_rate|ser_amt|
+--------------------+------------+------------+---+-------+-------+
|    TWO WHEELER TUBE| STEVEN KIN

## <a id='q53'></a>Q.53 specify the vehicles owners who’s tube damaged.

In [102]:
# Q.53 specify the vehicles owners who’s tube damaged.
# -- DataFrame API Solution --
ser_det_df.where(F.col("typ_ser") == 'TUBE DAMAGED').join(customer_df, "cid").select("cname").distinct().show()
# -- Spark SQL Solution --
spark.sql("""
    SELECT DISTINCT c.cname
    FROM customer_table c
    JOIN ser_det_table s ON c.cid = s.cid
    WHERE s.typ_ser = 'TUBE DAMAGED'
""").show()

+-----------+
|      cname|
+-----------+
|cyona blake|
|ANDRU SYMON|
+-----------+

+-----------+
|      cname|
+-----------+
|cyona blake|
|ANDRU SYMON|
+-----------+



## <a id='q54'></a>Q.54 Specify the details who have taken full service.

In [103]:
# Q.54 Specify the details who have taken full service.
# -- DataFrame API Solution --
ser_det_df.where(F.col("typ_ser") == 'FULL SERVICING').join(customer_df, "cid").select("cname", "cadd", "veh_no", "ser_date").show()
# -- Spark SQL Solution --
spark.sql("""
    SELECT c.cname, c.cadd, s.veh_no, s.ser_date
    FROM customer_table c
    JOIN ser_det_table s ON c.cid = s.cid
    WHERE s.typ_ser = 'FULL SERVICING'
""").show()

+----------+---------+---------+---------+
|     cname|     cadd|   veh_no| ser_date|
+----------+---------+---------+---------+
|JOHN SMITH|NEW JERSI|MH16U5713|04-MAR-11|
|JOHN SMITH|NEW JERSI|MH16U5713|05-MAY-11|
+----------+---------+---------+---------+

+----------+---------+---------+---------+
|     cname|     cadd|   veh_no| ser_date|
+----------+---------+---------+---------+
|JOHN SMITH|NEW JERSI|MH16U5713|04-MAR-11|
|JOHN SMITH|NEW JERSI|MH16U5713|05-MAY-11|
+----------+---------+---------+---------+



## <a id='q55'></a>Q.55 Select the employees who have not worked yet and left the job.

In [104]:
# Q.55 Select the employees who have not worked yet and left the job.
# -- DataFrame API Solution --
serviced_employees = ser_det_df.select("eid").distinct()
employee_df.where(F.col("edol").isNotNull()).join(serviced_employees, "eid", "left_anti").select("ename").show()
# -- Spark SQL Solution --
spark.sql("""
    SELECT e.ename
    FROM employee_table e
    WHERE e.edol IS NOT NULL AND e.eid NOT IN (SELECT eid FROM ser_det_table)
""").show()

+-----------+
|      ename|
+-----------+
|SHERI GOMES|
+-----------+

+-----------+
|      ename|
+-----------+
|SHERI GOMES|
+-----------+



## <a id='q56'></a>Q.56 Select employee who have worked first ever.

In [105]:
# Q.56 Select employee who have worked first ever.
# -- DataFrame API Solution --
first_service_date = ser_det_df.select(F.min("ser_date")).first()[0]
ser_det_df.where(F.col("ser_date") == first_service_date).join(employee_df, "eid").select("ename").show()
# -- Spark SQL Solution --
spark.sql("""
    SELECT e.ename
    FROM employee_table e
    JOIN ser_det_table s ON e.eid = s.eid
    WHERE s.ser_date = (SELECT MIN(ser_date) FROM ser_det_table)
""").show()

+-----------+
|      ename|
+-----------+
|STEVEN KING|
+-----------+

+-----------+
|      ename|
+-----------+
|STEVEN KING|
+-----------+



## <a id='q57'></a>Q.57 Display all records falling in odd date.

In [106]:
# Q.57 Display all records falling in odd date.
# -- DataFrame API Solution --
ser_det_df.where(F.dayofmonth(F.to_date(F.col("ser_date"), "dd-MMM-yy")) % 2 != 0).show()
# -- Spark SQL Solution --
spark.sql("SELECT * FROM ser_det_table WHERE MOD(DAY(TO_DATE(ser_date, 'dd-MMM-yy')), 2) != 0").show()

+----+----+----+----+------------+----------+--------------+---------+---+-------+------+----+-------+----+-----+
| sid| cid| eid|spid|    type_veh|    veh_no|       typ_ser| ser_date|qty|sp_rate|sp_amt|sp_g|ser_amt|comm|total|
+----+----+----+----+------------+----------+--------------+---------+---+-------+------+----+-------+----+-----+
|6004|1002|3002|4002| TWO WHEELER| MH16U5713|FULL SERVICING|05-MAY-11|1.0|    400|   400|  52|    300|  50|  752|
|6005|1004|3001|4009| TWO WHEELER| MH14PA335|         COLOR|21-OCT-11|2.5|    340|   850| 119|    500| 150| 1469|
|6006|1006|3001|4009| TWO WHEELER| MH12WE334|         COLOR|01-DEC-11|2.5|    340|   850| 119|    500| 150| 1469|
|6007|1007|3001|4001|FOUR WHEELER|MH17BB1345|  TUBE DAMAGED|01-JAN-12|1.0|    250|   250|  35|    150|   0|  435|
+----+----+----+----+------------+----------+--------------+---------+---+-------+------+----+-------+----+-----+

+----+----+----+----+------------+----------+--------------+---------+---+-------+-----

## <a id='q58'></a>Q.58 Display all records falling in even date.

In [107]:
# Q.58 Display all records falling in even date.
# -- DataFrame API Solution --
ser_det_df.where(F.dayofmonth(F.to_date(F.col("ser_date"), "dd-MMM-yy")) % 2 == 0).show()
# -- Spark SQL Solution --
spark.sql("SELECT * FROM ser_det_table WHERE MOD(DAY(TO_DATE(ser_date, 'dd-MMM-yy')), 2) = 0").show()

+----+----+----+----+-----------+----------+--------------+---------+---+-------+------+----+-------+----+-----+
| sid| cid| eid|spid|   type_veh|    veh_no|       typ_ser| ser_date|qty|sp_rate|sp_amt|sp_g|ser_amt|comm|total|
+----+----+----+----+-----------+----------+--------------+---------+---+-------+------+----+-------+----+-----+
|6001|1001|3001|4001|TWO WHEELER|MH15CA3228|  TUBE DAMAGED|02-JAN-11|1.0|    250|   250|  35|     50|   0|  335|
|6002|1002|3002|4002|TWO WHEELER| MH16U5713|FULL SERVICING|04-MAR-11|1.0|    400|   400|  52|    300|  50|  752|
|6003|1003|3004|4005|TWO WHEELER|MH12PQ1313|   CLUTCH WIRE|22-AUG-11|1.0|    129|   129|   0|     10|   0|  139|
+----+----+----+----+-----------+----------+--------------+---------+---+-------+------+----+-------+----+-----+

+----+----+----+----+-----------+----------+--------------+---------+---+-------+------+----+-------+----+-----+
| sid| cid| eid|spid|   type_veh|    veh_no|       typ_ser| ser_date|qty|sp_rate|sp_amt|sp_g|se

## <a id='q59'></a>Q.59 Display the vendors whose material is not yet used.

In [108]:
# Q.59 Display the vendors whose material is not yet used.
# -- DataFrame API Solution --
used_spids = ser_det_df.select("spid").distinct()
purchased_but_unused_spids = purchase_df.join(used_spids, "spid", "left_anti").select("vid").distinct()
purchased_but_unused_spids.join(vendor_df, "vid").select("vname").show()
# -- Spark SQL Solution --
spark.sql("""
    SELECT v.vname
    FROM vendor_table v
    WHERE v.vid IN (
        SELECT p.vid FROM purchase_table p
        WHERE p.spid NOT IN (SELECT spid FROM ser_det_table)
    )
""").show()

+-------------+
|        vname|
+-------------+
|  KIRAN PATIL|
|SWAPNIL THETE|
| KARAN SINTRE|
| RAM KULKARNI|
+-------------+

+-------------+
|        vname|
+-------------+
|  KIRAN PATIL|
|SWAPNIL THETE|
| KARAN SINTRE|
| RAM KULKARNI|
+-------------+



## <a id='q60'></a>Q.60 Difference between purchase date and used date of spare part.

In [109]:
# Q.60 Difference between purchase date and used date of spare part.
# -- DataFrame API Solution --
ser_det_df.join(purchase_df, "spid").join(sparepart_df, "spid").withColumn("days_difference", F.datediff(F.to_date(F.col("ser_date"), "dd-MMM-yy"), F.to_date(F.col("pdate"), "dd-MMM-yy"))).select("sid", "spname", "pdate", "ser_date", "days_difference").show()
# -- Spark SQL Solution --
spark.sql("""
    SELECT s.sid, sp.spname, p.pdate, s.ser_date,
           DATEDIFF(TO_DATE(s.ser_date, 'dd-MMM-yy'), TO_DATE(p.pdate, 'dd-MMM-yy')) AS days_difference
    FROM ser_det_table s
    JOIN purchase_table p ON s.spid = p.spid
    JOIN sparepart_table sp ON s.spid = sp.spid
""").show()

# Stop the Spark Session (uncomment to stop at the end)
# spark.stop()

+----+--------------------+---------+---------+---------------+
| sid|              spname|    pdate| ser_date|days_difference|
+----+--------------------+---------+---------+---------------+
|6007|    TWO WHEELER TUBE|01-JAN-11|01-JAN-12|            365|
|6001|    TWO WHEELER TUBE|01-JAN-11|02-JAN-11|              1|
|6004|TWO WHEELER ENGIN...|02-MAR-11|05-MAY-11|             64|
|6002|TWO WHEELER ENGIN...|02-MAR-11|04-MAR-11|              2|
|6003|TWO WHEELER CLUTC...|22-AUG-11|22-AUG-11|              0|
|6003|TWO WHEELER CLUTC...|12-OCT-11|22-AUG-11|            -51|
|6003|TWO WHEELER CLUTC...|15-OCT-11|22-AUG-11|            -54|
|6006|        WHITE COLOUR|20-OCT-11|01-DEC-11|             42|
|6005|        WHITE COLOUR|20-OCT-11|21-OCT-11|              1|
+----+--------------------+---------+---------+---------------+

+----+--------------------+---------+---------+---------------+
| sid|              spname|    pdate| ser_date|days_difference|
+----+--------------------+---------+--